In [ ]:
import panel as pn
import datetime as dt
import pandas as pd
pn.extension()

# Wired Web Component Example

The [Wired JS](https://wiredjs.com/) widgets provides a set of common UI elements with a hand-drawn, sketchy look. 
These can be used for wireframes, mockups, or just the fun hand-drawn look. 

<img src="https://camo.githubusercontent.com/b399ff5eef4c55afd9ad6c3195b1294e41afe3b9/68747470733a2f2f7062732e7477696d672e636f6d2f6d656469612f44595837782d76577341456863544a2e6a7067" style="width:67%;border:solid">

You might combine the Wired components with 

- sketchy plots generated by [matplotlib.pyplot.xkcd](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.xkcd.html) or
- sketchy drawings via [rough.js](https://roughjs.com/)

You can also find inspiration at the [Wired JS Showcase](https://wiredjs.com/showcase.html).

## Implementation

We implement the Wired components using the `pn.pane.WebComponent`. 

In [ ]:
"""Implementation of the Wired WebComponents"""
import ast
import datetime
import json
from typing import Set, Optional

import param

from panel.pane import WebComponent

JS_FILES = {
    "webcomponents-loader": "https://unpkg.com/@webcomponents/webcomponentsjs@2.2.7/webcomponents-loader.js",
    "wired-bundle": "https://wiredjs.com/dist/showcase.min.js",
}

ELEVATION_DEFAULT = 0
ELEVATION_BOUNDS = (0, 10)

### The Wired Javascript library

In [ ]:
%%HTML
<script src="https://unpkg.com/@webcomponents/webcomponentsjs@2.2.7/webcomponents-loader.js"></script>
<script src="https://wiredjs.com/dist/showcase.min.js"></script>

### MWC Icons

The wired components uses the MWC Icons in some of the components and list of `MWC_ICONS` is also defined.

Only a short list is provided here for illustration. You can find the full list [here](https://github.com/google/material-design-icons/blob/master/iconfont/codepoints)

In [ ]:
MWC_ICONS = [
    "3d_rotation",
    "ac_unit",
    "access_alarm",
    "access_alarms",
    "access_time",
    "accessibility",
    "accessible",
    "favorite",
]

#### Section Helper Functionality

The `section` shows the `component` together with a title, message, it's parameters and a divider.

In [ ]:
def section(component, message=None, show_html=False):
        title = "## " + str(type(component)).split(".")[1][:-2]

        parameterset = set(component._child_parameters())
        if show_html:
            parameterset.add("html")
        for parameter in component.parameters_to_watch:
            parameterset.add(parameter)

        parameters = list(parameterset)
        if message:
            return (
                title,
                component,
                pn.Param(component, parameters=parameters),
                pn.pane.Markdown(message),
                pn.layout.Divider(),
            )
        return pn.Column(title, component, pn.Param(component, parameters=parameters), pn.layout.Divider())

#### Base

The `WiredBase` provides a `Base` class that many of the components will inherit from

In [ ]:
class WiredBase(WebComponent):
    """Inherit from this class"""

    def __init__(self, **params):
        if not self.param.attributes_to_watch.default:
            self.param.attributes_to_watch.default = {}
        self.attributes_to_watch["disabled"] = "disabled"

        super().__init__(**params)

    def _child_parameters(self):
        parameters = super()._child_parameters()
        parameters.add("disabled")
        return parameters

## Button

In [ ]:
class Button(WiredBase):
    """A Wired RadioButton

    - You can set the `text` shown via the `name` parameter.
    """

    html = param.String("<wired-button></wired-button>")
    attributes_to_watch = param.Dict({"elevation": "elevation"})
    events_to_watch = param.Dict(default={"click": "clicks"})
    parameters_to_watch = param.List(["name"])

    clicks = param.Integer()
    elevation = param.Integer(ELEVATION_DEFAULT, bounds=ELEVATION_BOUNDS)

    def __init__(self, **params):
        if "height" not in params:
            params["height"]=40
        super().__init__(**params)

    def _get_html_from_parameters_to_watch(self, **params) -> str:
        return f"<wired-button>{params['name']}</wired-button>"

#### Button Example

In [ ]:
button = Button(Button(name="Allo Panel")
section(button)

##### Button Example with `pn.Param`

In [ ]:
class ButtonParameterizedClass(param.Parameterized):
    action = param.Action()
    clicks = param.Integer(default=0)
    
    def __init__(self, **params):
        super().__init__(**params)
        
        self.action = self._update
    
    def _update(self, event):
        self.clicks += 1
    
button_parameter_class = ButtonParameterizedClass()
pn.Column(
    pn.Param(button_parameter_class, widgets={"action": Button})
)

In [ ]:








class Checkbox(WiredBase):
    html = param.String("<wired-checkbox></wired-checkbox>")
    properties_to_watch = param.Dict({"checked": "value"})
    parameters_to_watch = param.List(["name"])

    value = param.Boolean()

    def __init__(self, **params):
        if "height" not in params:
            params["height"]=40

        super().__init__(**params)

    def _get_html_from_parameters_to_watch(self, **params) -> str:
        return f"<wired-checkbox>{params['name']}</wired-checkbox>"

class DatePicker(WiredBase):
    component_type = param.String("inputgroup")
    html = param.String(
        '<wired-calendar initials="" role="dialog tabindex="0">Button</wired-calendar>'
    )
    attributes_to_watch = param.Dict(
        {
            "elevation": "elevation",
            "firstdate": "firstdate",
            "lastdate": "lastdate",
            "locale": "locale",
        }
    )
    properties_to_watch = param.Dict({"selected": "selected"})
    events_to_watch = param.Dict(default={"selected": "selects"})

    elevation = param.Integer(ELEVATION_DEFAULT, bounds=ELEVATION_BOUNDS)
    # Todo: Support more datatime datahandling instead of strings if possible.
    firstdate = param.String(
        doc="""
    Example: firstdate="Apr 15, 2019"""
    )
    lastdate = param.String(
        doc="""
    Example: lastdate="Jul 15, 2019"""
    )
    locale = param.ObjectSelector("en", objects=["en", "fr", "de"])
    selected = param.String(
        doc="""
    Example: selected="Jul 4, 2019"""
    )
    selects = param.Integer(bounds=(0, None))
    value = param.Date(default=None)
    start = param.Date(default=None)
    end = param.Date(default=None)

    def __init__(self, min_height=340, min_width=300, **params):
        super().__init__(min_height=min_height, min_width=min_width, **params)

    @staticmethod
    def _to_date(value: Optional[str]) -> Optional[datetime.date]:
        """Converts a date string to a date

        Parameters
        ----------
        value : str
            The str date value
        """
        if value:
            return datetime.datetime.strptime(value, "%b %d, %Y").date()
        return None

    @staticmethod
    def _to_string(value: datetime.date) -> str:
        """Converts a date to a string

        Parameters
        ----------
        value : date
            The date value to convert
        """
        if value:
            return value.strftime("%b %e, %Y").replace("  ", " ")
        return None

    @param.depends("selected", watch=True)
    def _set_value(self):
        value = self._to_date(self.selected)
        if value != self.value:
            self.value = value

    @param.depends("value", watch=True)
    def _set_selected(self):
        selected = self._to_string(self.value)
        if selected != self.selected:
            self.selected = selected

    @param.depends("firstdate", watch=True)
    def _set_start(self):
        start = self._to_date(self.firstdate)
        if start != self.start:
            self.start = start

    @param.depends("start", watch=True)
    def _set_firstdate(self):
        firstdate = self._to_string(self.start)
        if firstdate != self.firstdate:
            self.firstdate = firstdate

    @param.depends("lastdate", watch=True)
    def _set_end(self):
        end = self._to_date(self.lastdate)
        if end != self.end:
            self.end = end

    @param.depends("end", watch=True)
    def _set_lastdate(self):
        lastdate = self._to_string(self.end)
        if lastdate != self.lastdate:
            self.lastdate = lastdate


# @Philppfr: The Dialog is really a layout
# I would like to support WebComponent layouts in general.
# The user should be able to include panes and widgets like buttons in the Dialog.
# How would I do that?
class Dialog(WebComponent):
    html = param.String("<wired-dialog></wired-checkbox>")
    attributes_to_watch = param.Dict({"open": "is_open"})
    parameters_to_watch = param.List(["text"])

    is_open = param.Boolean(default=False)
    text = param.String()

    def __init__(self, **params):
        super().__init__(**params)

    def _get_html_from_parameters_to_watch(self, **params) -> str:
        return f"<wired-dialog>{params['text']}</wired-dialog>"


class Divider(WebComponent):
    html = param.String("<wired-divider></wired-divider>")

    def __init__(self, min_height=20, **params):
        super().__init__(min_height=min_height, **params)

    attributes_to_watch = param.Dict({"elevation": "elevation"})
    elevation = param.Integer(ELEVATION_DEFAULT, bounds=ELEVATION_BOUNDS)


class Fab(WiredBase):
    html = param.String("<wired-fab><mwc-icon>favorite</mwc-icon></wired-fab>")
    parameters_to_watch = param.List(["icon"])

    icon = param.ObjectSelector(
        "favorite",
        objects=MWC_ICONS,
        doc="""
    The name of an `mwc-icon <https://github.com/material-components/material-components-web-components/tree/master/packages/icon>`_
    """,
    )

    def __init__(
        self, min_height=40, **params,
    ):
        super().__init__(min_height=min_height, **params)

    def _get_html_from_parameters_to_watch(self, **params) -> str:
        return f"<wired-fab><mwc-icon>{params['icon']}</mwc-icon></wired-fab>"

# Issue: Value is not set on construction. See
# https://github.com/wiredjs/wired-elements/issues/121#issue-573516963
# Todo: Add slider value to label
class FloatSlider(WebComponent):
    component_type = param.String("inputgroup")
    html = param.String("<wired-slider style='width: 100%;height:100%'></wired-slider>")
    attributes_to_watch = param.Dict({"min": "start", "max": "end", "step": "step"})
    properties_to_watch = param.Dict({"input.value": "value"})
    events_to_watch = param.Dict({"change": None})

    def __init__(self, min_height=40, **params):
        super().__init__(min_height=min_height, **params)

    start = param.Number(0.0)
    end = param.Number(1.0)
    step = param.Number(0.1)
    value = param.Number(default=0.0)

class IconButton(WiredBase):
    html = param.String("<wired-icon-button><mwc-icon>favorite</mwc-icon><wired-icon-button>")
    parameters_to_watch = param.List(["icon"])
    events_to_watch = param.Dict(default={"click": "clicks"})

    icon = param.ObjectSelector(
        "favorite",
        objects=MWC_ICONS,
        doc="""
    The name of an `mwc-icon <https://github.com/material-components/material-components-web-components/tree/master/packages/icon>`_
    """,
    )
    clicks = param.Integer()

    def __init__(
        self, min_height=40, **params,
    ):
        super().__init__(min_height=min_height, **params)

    def _get_html_from_parameters_to_watch(self, **params) -> str:
        return f"<wired-icon-button><mwc-icon>{params['icon']}</mwc-icon></wired-icon-button>"


class Image(WebComponent):
    """The wired-image element"""

    html = param.String('<wired-image style="width:100%;height:100%"></wired-image>')
    attributes_to_watch = param.Dict({"elevation": "elevation", "src": "object", "alt": "alt_text"})

    # @Philippfr: How do I handle height and width in general in the .ts model?
    def __init__(self, height=100, **params):
        super().__init__(height=height, **params)

    object = param.String(default=None, doc="""Currently only an url is supported""")
    alt_text = param.String(default=None)
    elevation = param.Integer(ELEVATION_DEFAULT, bounds=ELEVATION_BOUNDS)

# Issue: Value is not set on construction. See
# https://github.com/wiredjs/wired-elements/issues/121#issue-573516963
# Todo: Add slider value to label
class IntSlider(FloatSlider):

    def __init__(self, min_height=40, **params):
        super().__init__(min_height=min_height, **params)

    start = param.Integer(0)
    end = param.Integer(1)
    step = param.Integer(1)
    value = param.Integer(0)

class Link(WebComponent):
    html = param.String("<wired-link></wired-link>")
    attributes_to_watch = param.Dict({"href": "href", "target": "target"})
    parameters_to_watch = param.List(["text"])

    href = param.String()
    target = param.ObjectSelector("_blank", objects=["_self", "_blank", "_parent", "_top"])
    text = param.String()

    def __init__(self, **params):
        super().__init__(**params)

    def _get_html_from_parameters_to_watch(self, **params) -> str:
        return f"<wired-link>{params['text']}</wired-link>"

class LiteralInput(WiredBase):
    component_type = param.String("inputgroup")
    html = param.String("""<wired-input style="width:100%"></wired-input>""")
    attributes_to_watch = param.Dict(
        {
            "placeholder": "placeholder",
        }
    )
    properties_to_watch = param.Dict({"textInput.value": "value"})
    events_to_watch = param.Dict({"change": None})

    # @Philippff. I sthis the right place to define height? And what about width?
    def __init__(self, min_height=60, **params):
        # Hack: To solve https://github.com/wiredjs/wired-elements/issues/123
        if "value" in params:
            self.param.html.default = f'<wired-input value="{params["value"]}" style="width:100%;height:100%"></wired-input>'
        elif self.param.value.default:
            self.param.html.default = f'<wired-input value="{self.param.value.default}" style="width:100%;height:100%"></wired-input>'

        super().__init__(min_height=min_height, **params)

    placeholder = param.String(default="Enter Value")
    value = param.Parameter()
    type = param.ClassSelector(default=None, class_=(type, tuple),
                               is_instance=True)
    serializer = param.ObjectSelector(default='ast', objects=['ast', 'json'], doc="""
       The serialization (and deserialization) method to use. 'ast'
       uses ast.literal_eval and 'json' uses json.loads and json.dumps.
    """)

    def _set_type(self):
        if not self.value:
            class_ = (type,tuple)



    def _handle_properties_last_change(self, event):
        if "textInput.value" in event.new:
            value = event.new["textInput.value"]
            if not value or not isinstance(value, str):
                pass
            elif self.serializer == 'json':
                value = json.loads(value)
            else:
                value = ast.literal_eval(value)
            if value != self.value:
                self.value=value
        else:
            super()._handle_properties_last_change(event)


    def _handle_parameter_property_change(self, event):
        if event.name=="value":
            value = event.new

            if not value or isinstance(value, str):
                pass
            else:
                if self.serializer == 'json':
                    value = json.dumps(value)
                else:
                    value = repr(value)
            properties_last_change = {"textInput.value": value}

            if properties_last_change != self.properties_last_change:
                self.properties_last_change = properties_last_change
        else:
            super()._handle_parameter_property_change(event)


# Todo: Implement Wired wired-listbox


class Progress(WebComponent):
    html = param.String("<wired-progress></wired-progress>")
    attributes_to_watch = param.Dict({"value": "value", "percentage": "percentage", "max": "max"})

    def __init__(self, min_height=40, **params):
        super().__init__(min_height=min_height, **params)

        if "max" in params:
            self._handle_max_changed()

    value = param.Integer(None, bounds=(0, 100))
    max = param.Integer(100, bounds=(0, None))
    percentage = param.Boolean()

    @param.depends("max", watch=True)
    def _handle_max_changed(self):
        self.param.value.bounds = (0, self.max)


class RadioButton(WebComponent):
    """A Wired RadioButton"""

    html = param.String("<wired-radio>Radio Button</wired-radio>")
    properties_to_watch = param.Dict({"checked": "value"})
    parameters_to_watch = param.List(["name"])

    value = param.Boolean(default=False)

    def _get_html_from_parameters_to_watch(self, **params) -> str:
        return f"<wired-radio>{params['name']}</wired-radio>"

# Todo: Implement RadioBoxGroup
# Todo: Implement PasswordInput


class SearchInput(WiredBase):
    html = param.String("<wired-search-input></wired-search-input>")
    attributes_to_watch = param.Dict({"placeholder": "placeholder", "autocomplete": "autocomplete"})
    properties_to_watch = param.Dict({"textInput.value": "value"})
    events_to_watch = param.Dict({"input": None})

    def __init__(self, min_height=40, **params):
        if "value" in params:
            self.param.html.default = f'<wired-search-input value="{params["value"]}" style="width:100%;height:100%"></wired-search-input>'
        elif self.param.value.default:
            self.param.html.default = f'<wired-search-input value="{self.param.value.default}" style="width:100%;height:100%"></wired-search-input>'

        super().__init__(min_height=min_height, **params)

    placeholder = param.String("")
    value = param.String()
    autocomplete = param.ObjectSelector("off", objects=["on", "off"])


# Todo: Implement Tabs. It's really a layout. Don't yet know how to support this.

# @Philippfr: Should we merge the wired Progress and wired ProgressSpinner into one with
# functionality similar to the Panel Progress?
class ProgressSpinner(WebComponent):
    html = param.String("<wired-spinner></wired-spinner>")
    attributes_to_watch = param.Dict({"spinning": "active", "duration": "duration"})

    active = param.Boolean(default=True)
    duration = param.Integer(default=1000, bounds=(1, 10000))

    def __init__(self, min_height=40, **params):
        super().__init__(min_height=min_height, **params)

class TextAreaInput(WiredBase):
    component_type = param.String("inputgroup")
    html = param.String('<wired-textarea placeholder="Enter text"></wired-textarea>')
    attributes_to_watch = param.Dict({"placeholder": "placeholder"})
    properties_to_watch = param.Dict({"textareaInput.value": "value", "rows": "rows", "maxlength": "max_length"})
    events_to_watch = param.ObjectSelector(
        {"change": None},
        objects=[{"change": None}, {"input": None}],
        doc="""
    The event(s) to watch. When the event(s) are catched the js model properties are checked and
    any changed values are sent to the python model. The event can be
    - `change` (when done) or
    - `input` (for every character change)
    """,
    )

    placeholder = param.String("")
    value = param.String()
    rows = param.Integer(2, bounds=(1, 100))
    max_length = param.Integer(default=5000)

    def __init__(self, **params):
        # if "value" in params:
        #     self.param.html.default = f'<wired-textarea value="{params["value"]}"></wired-textarea>'
        # elif self.param.value.default:
        #     self.param.html.default = f'<wired-textarea value="{self.param.value.default}"></wired-textarea>'

        super().__init__(**params)

        if not "min_height" in params:
            self._set_height()

    @param.depends("rows", "disabled", watch=True)
    def _set_height(self):
        height = 20 + 19 * self.rows
        if self.disabled:
            height += 4
        if height != self.height:
            self.height = height


class TextInput(WiredBase):
    component_type = param.String("inputgroup")
    html = param.String("""<wired-input style="width:100%;height:100%"></wired-input>""")
    attributes_to_watch = param.Dict(
        {
            "placeholder": "placeholder",
            "type": "type_",
            # "min": "start",
            # "max": "end",
            # "step": "step",
        }
    )
    properties_to_watch = param.Dict({"textInput.value": "value"})
    events_to_watch = param.Dict({"change": None})

    # @Philippff. I sthis the right place to define height? And what about width?
    def __init__(self, min_height=50, **params):
        # Hack: To solve https://github.com/wiredjs/wired-elements/issues/123
        if "value" in params:
            self.param.html.default = f'<wired-input value="{params["value"]}" style="width:100%;height:100%"></wired-input>'
        elif self.param.value.default:
            self.param.html.default = f'<wired-input value="{self.param.value.default}" style="width:100%;height:100%"></wired-input>'

        super().__init__(min_height=min_height, **params)

    placeholder = param.String(default="")
    type_ = param.ObjectSelector("", objects=["", "password"])
    value = param.String()
    # start = param.Integer(None)
    # end = param.Integer(None)
    # step = param.Parameter(None)

class Toggle(WiredBase):
    html = param.String("<wired-toggle></wired-toggle>")
    properties_to_watch = param.Dict({"checked": "value"})
    events_to_watch = param.Dict({"change": None})

    def __init__(self, min_height=20, **params):
        super().__init__(min_height=min_height, **params)

    value = param.Boolean(False)


class Select(WebComponent):
    # Todo: Implement api for added wired-combo-items to the innerhtml
    # Todo: The selected attribute/ parameter is not updated. Fix this
    component_type = param.String("inputgroup")
    html = param.String("""<wired-combo></wired-combo>""")
    properties_to_watch = param.Dict({"selected": "value"})
    events_to_watch = param.Dict(default={"selected": "selects"})
    parameters_to_watch = param.List(["options"])

    def __init__(self, min_height=40, **params):
        super().__init__(min_height=min_height, **params)

        self._set_class_()

    value = param.Parameter()
    selects = param.Integer()
    options = param.ClassSelector(default=[], class_=(dict, list))

    def _get_html_from_parameters_to_watch(self, **params) -> str:
        options = params["options"]
        if not options:
            return """<wired-combo></wired-combo>"""

        innerhtml = []
        if isinstance(options, list):
            for obj in options:
                item = f'<wired-item value"{str(obj)}" role="option">{str(obj)}</wired-item>'
                innerhtml.append(item)
        if isinstance(options, dict):
            for key, value in options.items():
                item = f'<wired-item value"{str(key)}" role="option">{str(value)}</wired-item>'
                innerhtml.append(item)

        return f"""<wired-combo>{"".join(innerhtml)}</wired-combo>"""

    # @Phillipfr: Don't understand why this is nescessary. But get error if I don't do it.
    @param.depends("options", watch=True)
    def _set_class_(self):
        if isinstance(self.options, list):
            self.param.options.class_ = list
        if isinstance(self.options, dict):
            self.param.options.class_ = dict


class Video(WebComponent):
    html = param.String(
        """<wired-video autoplay="" playsinline="" muted="" loop="" style="height: 80%;" src="https://file-examples.com/wp-content/uploads/2017/04/file_example_MP4_480_1_5MG.mp4"></wired-video>"""
    )
    attributes_to_watch = param.Dict(
        {"autoplay": "autoplay", "playsinline": "playsinline", "loop": "loop", "src": "object"}
    )

    def __init__(self, min_height=250, margin=50, **params):
        super().__init__(min_height=min_height, **params)

    object = param.String(doc="""Currently only an url is supported""")
    autoplay = param.Boolean()
    playsinline = param.Boolean()
    muted = param.Boolean()
    loop = param.Boolean()


## Example

We develop an app with the two Tabs:

- **Wired View**: Show case the components
- **Param View**: Show case that the components can be used as widgets with `pn.Param`.

In [ ]:
def test_wired_view():
    js = """
<script src="https://unpkg.com/@webcomponents/webcomponentsjs@2.2.7/webcomponents-loader.js"></script>
<script src="https://wiredjs.com/dist/showcase.min.js"></script>
"""
    show_html = False

    js_pane = pn.pane.HTML(js)

    def section(component, message=None, show_html=show_html):
        title = "## " + str(type(component)).split(".")[1][:-2]

        parameterset = set(component._child_parameters())
        if show_html:
            parameterset.add("html")
        for parameter in component.parameters_to_watch:
            parameterset.add(parameter)

        parameters = list(parameterset)
        if message:
            return (
                title,
                component,
                pn.Param(component, parameters=parameters),
                pn.pane.Markdown(message),
                pn.layout.Divider(),
            )
        return (title, component, pn.Param(component, parameters=parameters), pn.layout.Divider())

    button = Button()
    check_box = Checkbox()
    date_picker = DatePicker()
    dialog = Dialog(text="Lorum Ipsum. Panel is awesome!")
    divider = Divider()
    fab = Fab()
    float_slider = FloatSlider()
    icon_button = IconButton()
    int_slider = IntSlider()
    image = Image(
        object="https://www.gstatic.com/webp/gallery/1.sm.jpg", height=200, width=300
    )
    link = Link(href="https://panel.holoviz.org/", text="Panel", target="_blank")
    # literal_input = LiteralInput(default={"a": 1, "b": "hello app world"})
    progress = Progress(value=50)
    progress_spinner = ProgressSpinner()
    radio_button = RadioButton()
    search_input = SearchInput()
    select = Select(
        html="""<wired-combo id="colorCombo" selected="red" role="combobox" aria-haspopup="listbox" tabindex="0" class="wired-rendered" aria-expanded="false"><wired-item value="red" aria-selected="true" role="option" class="wired-rendered">Red</wired-item><wired-item value="green" role="option" class="wired-rendered">Green</wired-item><wired-item value="blue" role="option" class="wired-rendered">Blue</wired-item></wired-combo>"""
    )
    # @Philippfr: How do I avoid the the pn.Param(slider, parameters=["value"]) to turn red when
    # using the slider? It seems the value needs to be rounded?
    text_area = TextAreaInput()
    text_input = TextInput()
    toggle = Toggle()
    video = Video(
        autoplay=True,
        loop=True,
        object="https://file-examples.com/wp-content/uploads/2017/04/file_example_MP4_480_1_5MG.mp4",
    )
    # section(select)
    return pn.Column(
        js_pane,
        *section(button),
        *section(check_box),
        *section(date_picker, "@Philippfr: I get a ValueError: Parameter 'value' only takes numeric values when selecting a date"),
        # *section(dialog, "@Philippfr: How do I add a close button and size the Dialog."),
        *section(divider),
        *section(fab),
        *section(
            float_slider,
            "@Philippfr: Currently an error is raised because the slider value is not rounded to 1 decimal",
        ),
        *section(icon_button),
        *section(image),
        *section(int_slider,),
        *section(
            link,
            "Normally you would just use the `<wired-link>` tag directly in your html or markdown text",
        ),
        # Todo: Fix AssertionError
        # *section(literal_input),
        *section(progress),
        *section(radio_button),
        *section(search_input, "@Philippfr. I cannot catch when the user clicks the x and clears the value"),
        *section(select),
        *section(progress_spinner),
        *section(text_area),
        *section(text_input),
        *section(toggle),
        *section(video),
        name = "Wired View",
    )

In [ ]:
def test_param_view():
    js = """
<script src="https://unpkg.com/@webcomponents/webcomponentsjs@2.2.7/webcomponents-loader.js"></script>
<script src="https://wiredjs.com/dist/showcase.min.js"></script>
"""
    js_pane = pn.pane.HTML(js)

    class BaseClass(param.Parameterized):
        x = param.Parameter(default=3.14, doc="X position")
        y = param.Parameter(default="Not editable", constant=True)
        string_value = param.String(default="str", doc="A string")
        num_int = param.Integer(50000, bounds=(-200, 100000))
        unbounded_int = param.Integer(23)
        float_with_hard_bounds = param.Number(8.2, bounds=(7.5, 10))
        float_with_soft_bounds = param.Number(0.5, bounds=(0, None), softbounds=(0, 2))
        unbounded_float = param.Number(30.01, precedence=0)
        hidden_parameter = param.Number(2.718, precedence=-1)
        integer_range = param.Range(default=(3, 7), bounds=(0, 10))
        float_range = param.Range(default=(0, 1.57), bounds=(0, 3.145))
        dictionary = param.Dict(default={"a": 2, "b": 9})

    class Example(BaseClass):
        """An example Parameterized class"""

        timestamps = []

        boolean = param.Boolean(True, doc="A sample Boolean parameter")
        color = param.Color(default="#FFFFFF")
        date = param.Date(
            dt.datetime(2017, 1, 1), bounds=(dt.datetime(2017, 1, 1), dt.datetime(2017, 2, 1))
        )
        dataframe = param.DataFrame(pd.util.testing.makeDataFrame().iloc[:3])
        select_string = param.ObjectSelector(default="yellow", objects=["red", "yellow", "green"])
        select_fn = param.ObjectSelector(default=list, objects=[list, set, dict])
        int_list = param.ListSelector(default=[3, 5], objects=[1, 3, 5, 7, 9], precedence=0.5)
        single_file = param.FileSelector(path="../../*/*.py*", precedence=0.5)
        multiple_files = param.MultiFileSelector(path="../../*/*.py?", precedence=0.5)
        record_timestamp = param.Action(
            lambda x: x.timestamps.append(dt.datetime.utcnow()),
            doc="""Record timestamp.""",
            precedence=0.7,
        )

    base = Example()
    parameters = [
        "x",
        "y",
        "string_value",
        "num_int",
        "unbounded_int", # Todo: Add Feature Request for unbounded int to Wired
        "float_with_hard_bounds",
        "float_with_soft_bounds",
        "unbounded_float", # Todo: Add Feature Request for unbounded int to
        "hidden_parameter",
        # "integer_range",  # Todo: Add Feature Request for Integer Range to Wired
        # "float_range",  # Todo: Add Feature Request for Float Range to Wired
        "dictionary",
        "boolean",
        # "color", # Todo: Add Feature Request for Color Picker to Wired
        "date",
        # "dataframe", # Todo: Add Feature Request for Table to Wired
        "select_string",
    ]
    widgets = {
        # Todo: Implement spinner to support Numbers
        # "x": TextInput,  # Todo: Find out why value is not shown on construction
        "y": TextInput,  # Todo: Find out why value is not shown on construction
        "string_value": TextInput,  # Todo: Find out why value is not shown on construction
        "num_int": IntSlider,  # Todo: Add value to label
        "unbounded_int": TextInput,  # Todo: Find out why unbounded_int does not use TextInput
        "float_with_hard_bounds": FloatSlider,  # Todo: Add value to label
        "float_with_soft_bounds": FloatSlider,  # Todo: Add value to label
        "unbounded_float": TextInput,  # Todo: Find out why unbounded_int does not use TextInput
        "dictionary": LiteralInput,
        "boolean": Checkbox,
        "date": DatePicker,
        "select_string": Select,
    }
    pn.Param(base, parameters=parameters, widgets=widgets)
    # @Philippfr: how do I get wired widgets to stretch_width automatically?
    return pn.Column(
        js_pane,
        pn.Row(
            pn.Param(base, parameters=parameters),
            pn.Param(base, parameters=parameters, widgets=widgets),
        ),
        name = "Param View",
    )

In [ ]:
pn.config.sizing_mode = "stretch_width"
wired_view = test_wired_view()
param_view = test_param_view()
app = pn.Column(
    pn.layout.Tabs(
        wired_view,
        param_view,
    ),
)
app

In [ ]:
def show(event):
    app.show()
button = pn.widgets.Button(name="Show", button_type="primary")
button.on_click(show)
button